In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer, ALL_PRETRAINED_CONFIG_ARCHIVE_MAP 
import faiss
import annoy
from tqdm.notebook import tqdm_notebook as tqdmnb

In [54]:
MODEL_TO_USE = 'bert-large-uncased'

In [5]:
%%time
args = pd.read_pickle(f'args_encoded_{MODEL_TO_USE}.pkl')
args.dropna(inplace=True)
args.reset_index(inplace=True, drop=True)
i = np.stack(args['input_ids'])
m = np.stack(args['attention_mask'])

CPU times: user 1min 40s, sys: 3.28 s, total: 1min 43s
Wall time: 1min 43s


In [56]:
args

,id,text,input_ids,attention_mask
0,c67482ba-2019-04-18T13:32:05Z-00000-000,My opponent forfeited every round. None of my ...,"[101, 2026, 7116, 2005, 21156, 2098, 2296, 246...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,c67482ba-2019-04-18T13:32:05Z-00001-000,How do you propose the school will fund your p...,"[101, 2129, 2079, 2017, 16599, 1996, 2082, 209...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,c67482ba-2019-04-18T13:32:05Z-00002-000,Schools have no compelling interest in providi...,"[101, 2816, 2031, 2053, 17075, 3037, 1999, 434...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,c67482ba-2019-04-18T13:32:05Z-00003-000,As a senior at my school. My group and I are f...,"[101, 2004, 1037, 3026, 2012, 2026, 2082, 1012...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,4d3d4471-2019-04-18T11:45:01Z-00000-000,The resolution used by Pro *assumes* that Aust...,"[101, 1996, 5813, 2109, 2011, 4013, 1008, 1598...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
555496,671509c8-2019-04-17T11:47:34Z-00067-000,"Seth Sandronsky. ""Rethinking Charter Schools""....","[101, 6662, 5472, 28212, 4801, 1012, 1000, 212...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
555497,671509c8-2019-04-17T11:47:34Z-00052-000,Charter schools proliferated too quickly to mo...,"[101, 6111, 2816, 4013, 15509, 9250, 2205, 285...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
555498,671509c8-2019-04-17T11:47:34Z-00037-000,Public schools can offer everything charter sc...,"[101, 2270, 2816, 2064, 3749, 2673, 6111, 2816...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
555499,671509c8-2019-04-17T11:47:34Z-00022-000,Charter schools are damaging private schools,"[101, 6111, 2816, 2024, 15011, 2797, 2816, 102...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [46]:
model = AutoModel.from_pretrained(MODEL_TO_USE)
gpu = torch.device('cuda')
model = model.cuda()

In [49]:
outputs = []
for batch_i, batch_m in tqdmnb(zip(np.array_split(i, 8679), np.array_split(m, 8679)), total=8679):
    tensor_i = torch.tensor(batch_i).to(gpu)
    tensor_m = torch.tensor(batch_m).to(gpu)
    with torch.no_grad():
        output = model(tensor_i, attention_mask=tensor_m)[0][:,0,:].cpu().numpy()
        outputs.append(output)

In [50]:
arg_representations = np.vstack(outputs)

In [53]:
%%time
np.save(f'./arg_representations_{MODEL_TO_USE}', arg_representations)

CPU times: user 0 ns, sys: 1.14 s, total: 1.14 s
Wall time: 1.27 s


In [52]:
arg_representations

array([[ 0.22579737, -0.07349378, -1.0474814 , ..., -0.9103607 ,
        -0.20819543, -0.17666322],
       [-0.1498387 , -0.4468132 , -0.99795085, ..., -1.2222996 ,
        -0.06732306,  0.26513594],
       [ 0.47659963, -1.0098528 , -0.6905463 , ..., -0.86388797,
        -0.432725  ,  0.15581201],
       ...,
       [-0.08755945, -0.49159864, -0.62298226, ..., -0.6357609 ,
        -0.41940513,  0.25646588],
       [ 0.0292035 , -0.8047588 , -0.3333864 , ..., -0.4762738 ,
        -0.7674418 ,  0.17478399],
       [-0.35893157, -0.23544711, -1.0336562 , ..., -0.09143204,
        -0.5442716 ,  0.21634527]], dtype=float32)